In [1]:
import numpy as np
import tensorflow as tf
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

from utils import f_props, load_mnist, tf_log
from layers import Dense
from optimizers import sgd, adamax

random_state = 42

In [46]:
tf.reset_default_graph() # グラフのリセット

# Step1. プレースホルダー・変数の設定
## placeholder: データを流し込む変数. データ毎に変わる
x = tf.placeholder(tf.float32, name='x')
t = tf.placeholder(tf.float32, name='t')

## Variable: 変数(重み). データ間で共有される
w = tf.Variable(0.0, name='w')
b = tf.Variable(0.0, name='b')
params = [w, b]

# Step2. グラフの構築
y = w*x + b

# Step3. 誤差関数の設定
cost = tf.reduce_mean((y - t)**2)

# Step4. 重みの更新則の設定
g_params = tf.gradients(cost, params) # 勾配の計算
updates = []
lr = np.float32(0.01)
eps = np.float32(1e-8)
for param, g_param in zip(params, g_params):
    g2_sum = tf.Variable(np.zeros(param.get_shape(), dtype='float32'), name='g2_sum')
    updates.append(g2_sum.assign(g2_sum + g_param**2))
    updates.append(param.assign(param - (lr/tf.sqrt(g2_sum + eps))*g_param))
train = tf.group(*updates)

# Step.5. 学習 (y = 2*x + 3)
data_X = np.array([0., 1., 2., 3., 4.])
data_y = np.array([3., 5., 7., 9., 11.])

sess = tf.Session()
sess.run(tf.global_variables_initializer()) # 重みの初期化
print(sess.run(params))
for i in range(1000):
    _cost, _, _g_params = sess.run([cost, train, g_params], feed_dict={x: data_X, t: data_y})
    break
#     if (i+1)%100==0:
#         print('iteration:: %d, cost:: %.3f' % (i+1, _cost))

# Step6. 予測
print('cost:', _cost)
print('params:', sess.run(params))
print('g_params:', _g_params)
print('tf.global:', sess.run(tf.global_variables()[2:]))
print()
# print('pred_y:', sess.run(y, feed_dict={x: [5]}))

_cost, _, _g_params = sess.run([cost, train, g_params], feed_dict={x: data_X, t: data_y})

print('cost:', _cost)
print('params:', sess.run(params))
print('g_params:', _g_params)
print('tf.global:', sess.run(tf.global_variables()[2:]))
print()

[0.0, 0.0]
cost: 57.0
params: [3600.0, 1400.0]
g_params: [-36.0, -14.0]
tf.global: [1296.0, 196.0]

cost: 9.97309e+07
params: [3586.4543, 1387.7242]
g_params: [48764.0, 17186.0]
tf.global: [2.377929e+09, 2.9535878e+08]



In [45]:
params

[<tf.Variable 'w:0' shape=() dtype=float32_ref>,
 <tf.Variable 'b:0' shape=() dtype=float32_ref>]

In [10]:
g_params

[<tf.Tensor 'gradients/mul_grad/Reshape:0' shape=() dtype=float32>,
 <tf.Tensor 'gradients/add_grad/Reshape_1:0' shape=() dtype=float32>]

In [11]:
updates

[<tf.Tensor 'Assign:0' shape=() dtype=float32_ref>,
 <tf.Tensor 'Assign_1:0' shape=() dtype=float32_ref>,
 <tf.Tensor 'Assign_2:0' shape=() dtype=float32_ref>,
 <tf.Tensor 'Assign_3:0' shape=() dtype=float32_ref>]

In [23]:
_g_params

[-36.0, -14.0]

In [25]:
tf.global_variables()

[<tf.Variable 'w:0' shape=() dtype=float32_ref>,
 <tf.Variable 'b:0' shape=() dtype=float32_ref>,
 <tf.Variable 'g2_sum:0' shape=() dtype=float32_ref>,
 <tf.Variable 'g2_sum_1:0' shape=() dtype=float32_ref>]